# Save Bus Data into Database
In this notebook, we take the cleaned bus data from CSV files and save it into a database.  

- We use **SQLite** because it is simple, file-based, and does not require a separate database server.  
- We also demonstrate saving data to **PostgreSQL**, which is a powerful relational database suitable for larger projects and multi-user access.


In [1]:
import pandas as pd
import sqlite3
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus


## Load CSV File


In [2]:
csv_path = "Data/All_Routes_Buses_Final.csv"

if not os.path.exists(csv_path):
    raise FileNotFoundError("CSV file not found. Run 01_web_scraping.ipynb first.")


**Read CSV into DataFrame**

Load the cleaned bus data from the CSV file into a Pandas DataFrame.  


In [3]:
df = pd.read_csv(csv_path)
df.head()


,Bus_ID,Departure,Arrival,Duration,Duration_Minutes,Seats,Single_Seats,Price,Onwards,Operator,Bus_Type,Rating,Rating_Count,Live_Tracking,Source,Destination
0,26409654,23:15,06:10,6h 55m,415,42,10,1250,Onwards,Jayavin Travels,A/C Seater / Sleeper (2+1),4.4,575,Yes,Bangalore,Chennai
1,44319378,21:55,04:30,6h 35m,395,36,12,585,Onwards,HYBUS,Bharat Benz A/C Sleeper (2+1),4.5,245,Yes,Bangalore,Chennai
2,36258160,21:35,05:00,7h 25m,445,24,8,1600,Onwards,PADMAVATHI TRAVELS,A/C Sleeper (2+1),4.3,797,Yes,Bangalore,Chennai
3,37971794,22:45,05:45,7h,420,25,6,1400,Onwards,Krish Travels,Bharat Benz A/C Seater /Sleeper (2+1),4.2,1377,Yes,Bangalore,Chennai
4,44319377,21:40,04:55,7h 15m,435,36,12,585,Onwards,HYBUS,Bharat Benz A/C Sleeper (2+1),4.5,166,Yes,Bangalore,Chennai


## Create SQLite Database

- Define the path for the SQLite database file (`bus_data.db`) inside the `data` folder.  
- Connect to the database. If the file does not exist, SQLite will create it automatically.


In [4]:
db_path = "data/bus_data.db"
conn = sqlite3.connect(db_path)


**Save Data to SQLite and Verify**

Save the `DataFrame` into the SQLite database as a table named `buses`.  



In [5]:
df.to_sql("buses", conn, if_exists="replace", index=False)
df_check = pd.read_sql("SELECT * FROM buses LIMIT 5", conn)
print("Data saved to SQLite database successfully")

print(df_check)
conn.close()



Data saved to SQLite database successfully
     Bus_ID Departure Arrival Duration  Duration_Minutes  Seats  Single_Seats  \
0  26409654     23:15   06:10   6h 55m               415     42            10   
1  44319378     21:55   04:30   6h 35m               395     36            12   
2  36258160     21:35   05:00   7h 25m               445     24             8   
3  37971794     22:45   05:45       7h               420     25             6   
4  44319377     21:40   04:55   7h 15m               435     36            12   

   Price  Onwards            Operator                               Bus_Type  \
0   1250  Onwards     Jayavin Travels             A/C Seater / Sleeper (2+1)   
1    585  Onwards               HYBUS          Bharat Benz A/C Sleeper (2+1)   
2   1600  Onwards  PADMAVATHI TRAVELS                      A/C Sleeper (2+1)   
3   1400  Onwards       Krish Travels  Bharat Benz A/C Seater /Sleeper (2+1)   
4    585  Onwards               HYBUS          Bharat Benz A/C Sleeper

## Save Data to PostgreSQL
 
- Set up PostgreSQL connection details.  
- Encode the password to handle special characters and create a SQLAlchemy engine.  
- Save the DataFrame into PostgreSQL as a table named `bus_routes` 


In [6]:
# Load your CSV file
df = pd.read_csv(csv_path)


# PostgreSQL connection details
db_user = "postgres"
db_password = "Ganesha123@!"
db_host = "localhost"
db_port = "5432"
db_name = "bus_data"  # your database in pgAdmin

# Encode password to handle special characters
db_password_encoded = quote_plus(db_password)

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password_encoded}@{db_host}:{db_port}/{db_name}")

# Store DataFrame in PostgreSQL
df.to_sql("bus_routes", engine, if_exists="replace", index=False)

# Verify by reading first 5 rows
df_check = pd.read_sql("SELECT * FROM bus_routes LIMIT 5", engine)
print("Data saved to PostgreSQL database successfully")
print(df_check)


Data saved to PostgreSQL database successfully
     Bus_ID Departure Arrival Duration  Duration_Minutes  Seats  Single_Seats  \
0  26409654     23:15   06:10   6h 55m               415     42            10   
1  44319378     21:55   04:30   6h 35m               395     36            12   
2  36258160     21:35   05:00   7h 25m               445     24             8   
3  37971794     22:45   05:45       7h               420     25             6   
4  44319377     21:40   04:55   7h 15m               435     36            12   

   Price  Onwards            Operator                               Bus_Type  \
0   1250  Onwards     Jayavin Travels             A/C Seater / Sleeper (2+1)   
1    585  Onwards               HYBUS          Bharat Benz A/C Sleeper (2+1)   
2   1600  Onwards  PADMAVATHI TRAVELS                      A/C Sleeper (2+1)   
3   1400  Onwards       Krish Travels  Bharat Benz A/C Seater /Sleeper (2+1)   
4    585  Onwards               HYBUS          Bharat Benz A/C Sle